# Описание проекта "Анализ поведения пользователей в мобильном приложении"

Менеджер продукта мобильного приложения «Ненужные вещи» хочет изучить поведение пользователей и сегментировать их для того, чтобы увеличить вовлечённость. Пользователи совершают много действий в приложении, они логируются и эти данные можно выгружать. В этих данных есть инсайты, которые позволят сервису стать лучшим приложением для продажи ненужных вещей.

## Цель исследования

Управление вовлеченностью клиентов (адаптация приложения по целевой и смежной аудитории) на основе данных о поведении пользователей.

**Получить на основе поведения пользователей гипотезы о том как можно было бы улучшить приложение с точки зрения пользовательского опыта.**

## Задачи

1. Проанализировать связь целевого события — просмотра контактов — и других действий пользователей.
- В разрезе сессий отобрать сценарии\паттерны, которые приводят к просмотру контактов
- Построить воронки по основным сценариям в разрезе уникальных пользователей
2. Оценить, какие действия чаще совершают те пользователи, которые
просматривают контакты.
- Рассчитать относительную частоту событий в разрезе двух групп пользователей:
  - группа пользователей, которые смотрели контакты `contacts_show`
  - группа пользователей, которые не смотрели контакты `contacts_show`

# Описание данных

Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений.
В датасете содержатся данные пользователей, впервые совершивших действия в приложении *после 7 октября 2019 года*.

**Колонки в `/datasets/mobile_sources.csv` :**
- userId — идентификатор пользователя,
- source — источник, с которого пользователь установил приложение.

**Колонки в `/datasets/mobile_dataset.csv` :** 
- `event.time` — время совершения,
- `user.id` — идентификатор пользователя, 
- `event.name` — действие пользователя.
  
  *Виды действий:*
  - `advert_open` — открыл карточки объявления,
  - `photos_show` — просмотрел фотографий в объявлении,
  - `tips_show` — увидел рекомендованные объявления,
  - `tips_click` — кликнул по рекомендованному объявлению,
  - `contacts_show` и `show_contacts` — посмотрел номер телефона, 
  - `contacts_call` — позвонил по номеру из объявления,
  - `map` — открыл карту объявлений,
  - `search_1` — `search_7` — разные действия, связанные с поиском по сайту, 
  - `favorites_add` — добавил объявление в избранное.

# Оглавление

1. [Загрузка данных](#start)
2. [Подготовка данных](#preparation)
  - 2.1. [Общая информация](#info) 
  - 2.2. [Пропуски](#nan) 
  - 2.3. [Типы данных](#dtype) 
  - 2.4. [Дубликаты](#duplicates)
3. [Исследовательский анализ](#eda)
  - 3.1. [Проверка уникальных значений](#unique) 
  - 3.2. [Распределение данных, статистика](#describe) 
4. [Основные вопросы исследования](#main)
  - 4.1. [Пользовательские сессии](#sessions) 
  - 4.2. [Сценарии использования](#path) 
  - 4.3. [Воронки и конверсия](#conversion) 
  - 4.4. [Частота событий](#frequency) 
5. [Проверка гипотез](#hypothesys)
  - 5.1. [Гипотеза 1](#h1)
  - 5.2. [Гипотеза 2](#h2)
6. [Выводы](#conclusion)

<a id="start"></a>
# Загрузка данных

In [1]:
# импорт библиотек
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import numpy as np


/var/folders/ff/y3_3yvs54qqbwrvfr1ln_p8m0000gn/T/ipykernel_3573/2869466893.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Избавиться от экспоненты в табличках
pd.options.display.float_format = '{:,.1f}'.format

In [3]:
# загружаем CSV-файлы с данными и сохраняем их в датафреймы
try:
    mobile_dataset = pd.read_csv('/datasets/mobile_dataset.csv')
    mobile_sources = pd.read_csv('/datasets/mobile_sources.csv')
except:
    mobile_dataset = pd.read_csv('../datasets/mobile_dataset.csv')
    mobile_sources = pd.read_csv('../datasets/mobile_sources.csv')

Проверим, что данные загрузились

In [4]:
# выводим первые 5 строк обоих датасетов
display(mobile_dataset.head(), mobile_sources.head())

,event.time,event.name,user.id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c


,userId,source
0,020292ab-89bc-4156-9acf-68bc2783f894,other
1,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
2,8c356c42-3ba9-4cb6-80b8-3f868d0192c3,yandex
3,d9b06b47-0f36-419b-bbb0-3533e582a6cb,other
4,f32e1e2a-3027-4693-b793-b7b3ff274439,google


<a id="preparation"></a>
# Подготовка и предобработка данных

Прежде, чем объединить датасеты, нужно поменять названия колонок и привести их к удобному виду.

In [5]:
# заменим точки на `_`
mobile_dataset.columns = mobile_dataset.columns.str.replace('.','_')

# переименуем столбец userId
mobile_sources.rename(columns={'userId':'user_id'}, inplace=True)


In [6]:
# выводим размерность датасетов
mobile_sources.shape, mobile_dataset.shape

((4293, 2), (74197, 3))

Теперь можем объединить датасеты

In [7]:
# Объединяем датасеты
data = mobile_dataset.merge(mobile_sources, on='user_id')
display(data.head())
data.shape

,event_time,event_name,user_id,source
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex


(74197, 4)

<a id="info"></a>
## Общая информация о датасете

Выведем основную информацию о получившемся датасете

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_time  74197 non-null  object
 1   event_name  74197 non-null  object
 2   user_id     74197 non-null  object
 3   source      74197 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


- В датасете 74197 строк, 4 колонки, они соответствуют описанию
- Пропуски (NaN) отсутствуют, уникальные значения проверим далее
- `event_time` следует привести к формату времени

<a id="nan"></a>
## Проверка на пропуски

В `info` было видно, что пропусков нет.

In [9]:
data.isna().sum()

event_time    0
event_name    0
user_id       0
source        0
dtype: int64

<a id="dtype"></a>
## Проверка и изменение типов данных

In [10]:
# приводим event_time к формату времени
data['event_time'] = pd.to_datetime(data['event_time'])
data.dtypes

event_time    datetime64[ns]
event_name            object
user_id               object
source                object
dtype: object

Остальные типы данных нас устраивают

<a id="duplicates"></a>
## Проверка на дубликаты (явные и неявные)

In [11]:
data.duplicated().sum()

0

Явных дубликатов нет. Могут считаться дубликатами быстрые последовательности событий в рамках одной секунды. Проверим их.

In [12]:
# Округляем 'event.time' до ближайшей секунды
time_duplicates = data.copy()
time_duplicates['event_time'] = time_duplicates['event_time'].dt.round('s')

# проверяем на дубликаты после округления
duplicates_after_rounding = time_duplicates[time_duplicates.duplicated()]

# Количество дубликатов
duplicates_after_rounding.value_counts(), time_duplicates.duplicated().sum()


(event_time           event_name     user_id                               source
 2019-10-27 20:00:07  photos_show    13140930-df18-4793-a230-7cca5c8813db  yandex    8
 2019-10-23 18:24:01  photos_show    62a5375a-eb94-4ed2-90ef-3d79d8e0c359  other     7
 2019-10-23 18:16:14  photos_show    62a5375a-eb94-4ed2-90ef-3d79d8e0c359  other     6
 2019-10-26 09:02:54  photos_show    2c0dfee3-b5c3-46eb-a92e-a469870fa1cc  other     6
 2019-10-17 23:45:02  contacts_show  136b7b37-2bd4-4718-b14a-e38bc3d6d112  other     6
                                                                                    ..
 2019-10-18 11:39:48  tips_show      f845d6b9-086d-4b35-b3db-478b44dbd0fc  other     1
 2019-10-18 12:47:40  photos_show    6d176cf2-81ec-4e7d-8b0a-da008c2d96fd  other     1
 2019-10-18 13:36:10  tips_show      b6cb89f2-0935-4e28-be32-50e439b8487f  other     1
 2019-10-18 14:05:47  tips_show      eb9c25db-4a56-4ad4-aead-35f7c39e7ebb  yandex    1
 2019-11-03 22:41:01  tips_show      16a5371c-15

In [13]:
duplicates_after_rounding['event_name'].value_counts()

event_name
photos_show      640
tips_show        163
contacts_show    148
map              121
advert_open       19
search_1          17
favorites_add      3
contacts_call      3
tips_click         3
search_3           1
Name: count, dtype: int64

Анализ дубликатов показал события, в которых пользователь осуществляет идентичные действия в одну и ту же секунду. 

- **Повторяющиеся действия**: такие как `photos_show`, `map`, и `tips_show`.
- **Возможные причины**:
  - **Быстрые последовательные действия**: Пользователи могли делать быстрые повторные действия, например, пролистывать фотографии, если интерфейс это позволяет.
  - **Поведение приложения**: Приложение может логировать несколько событий после одного пользовательского действия.
  - **Автоматические скрипты**: Также возможно наличие автоматических скриптов или ботов.

В контексте исследования повторные действия будут игнорироваться для построения шагов воронки небольшой длины. Дубликатов 1180 из 74 тыс. строк датасета, поэтому решаем их удалить.


In [14]:
data['event_time_rounded'] = time_duplicates['event_time'].dt.round('s')

data = data.drop_duplicates(subset=['event_name', 'event_time_rounded', 'user_id'])
data.duplicated(subset=['event_name', 'event_time_rounded', 'user_id']).sum()

0

<a id="eda"></a>
# Исследовательский анализ

Посмотрим на распределения значений и уникальные значения по столбцам

In [15]:
# Сводная статистика для числовых столбцов
display(data.describe())

,event_time,event_time_rounded
count,73079,73079
mean,2019-10-21 15:37:20.731564032,2019-10-21 15:37:20.732933120
min,2019-10-07 00:00:00.431357,2019-10-07 00:00:00
25%,2019-10-14 21:57:48.297969408,2019-10-14 21:57:48
50%,2019-10-22 00:21:49.770871040,2019-10-22 00:21:50
75%,2019-10-28 12:52:27.406280448,2019-10-28 12:52:27.500000
max,2019-11-03 23:58:12.532487,2019-11-03 23:58:13


Данные после 7 октября 2019 года - соответствует описанию

<a id="unique"></a>
## Проверка уникальных значений

In [16]:
# Уникальные значения категориальных столбцов
categorical_columns = data.select_dtypes(include='object').columns
for column in categorical_columns:
    unique_counts = data[column].value_counts()
    if len(unique_counts) <= 16:
        print(f"{column}:\n")
        display(unique_counts)

event_name:



event_name
tips_show        39892
photos_show       9372
advert_open       6145
contacts_show     4302
map               3760
search_1          3489
favorites_add     1414
search_5          1049
tips_click         811
search_4           701
contacts_call      538
search_3           521
search_6           460
search_2           324
search_7           222
show_contacts       79
Name: count, dtype: int64

source:



source
yandex    33854
google    20171
other     19054
Name: count, dtype: int64

`tips_show` по количеству - больше половины событий

значения `event_name` соответствуют описанию. так как `show_contacts` и `contacts_show` - это одно и то же событие, заменим значения

In [17]:
data['event_name'] = data['event_name'].replace({'show_contacts':'contacts_show'})
data['event_name'].value_counts()

event_name
tips_show        39892
photos_show       9372
advert_open       6145
contacts_show     4381
map               3760
search_1          3489
favorites_add     1414
search_5          1049
tips_click         811
search_4           701
contacts_call      538
search_3           521
search_6           460
search_2           324
search_7           222
Name: count, dtype: int64

<a id="main"></a>
# Основные вопросы исследования

<a id="sessions"></a>
## Выделение пользовательских сессий

Выделим пользовательские сессии основываясь на выбранном таймауте. Для этого будем создавать отдельный столбец с идентификатором сессии.

Для начала найдём распределение интервалов между действиями пользователей и попробуем выбрать интервал бездействия

In [18]:
# сортируем по пользователям и времени
sorted_data = data.sort_values(['user_id','event_time'])

# вычисляем время между действиями
sorted_data['time_diff'] = sorted_data.groupby('user_id')['event_time'].diff()

# переводим разницу во времени в минуты
sorted_data['time_diff_minutes'] = sorted_data['time_diff'].dt.total_seconds() / 60

# убираем пустые значения первого события
df = sorted_data.dropna(subset=['time_diff_minutes'])

# Create a boxplot
fig = px.box(df, y='time_diff_minutes', title='Развределение значений интервалов бездействия')

fig.update_layout(
    yaxis_title='Разница во времени в минутах'
)

# ограничим выбросы
fig.update_yaxes(range=[0, 10])

fig.show()

# вычислим процентиль значений
percentile_95 = df['time_diff_minutes'].quantile(0.95)
print(f"95й процентиль интервалов бездействия: {percentile_95} минут(ы)")

percentile_85 = df['time_diff_minutes'].quantile(0.85)
print(f"85й процентиль интервалов бездействия: {percentile_85} минут(ы)")

95й процентиль интервалов бездействия: 668.922550125 минут(ы)
85й процентиль интервалов бездействия: 6.423177475 минут(ы)


Межквартильный размах бездействия - 2,64 минуты. Для определения интервала, который примем за окончание сессии, можно 
1. использовать множитель для межквартильного размаха (часто используют 1.5), но может быть и больше
2. использовать определённый процентиль.

Мы посмотрели два процентиля - 95й и 85й. 85й лучше подходит, так как равен 2.4 межквартильных размаха

Возьмём 7 минут как значение интервала бездействия = окончания сессии

In [19]:
# находим 7 минутную разницу
g = (sorted_data.groupby('user_id')['event_time'].diff() > pd.Timedelta('7Min')).cumsum()

# создаём счётчик сессий
sorted_data['session_id'] = sorted_data.groupby(['user_id', g], sort=False).ngroup() + 1

sorted_data.head(10)

,event_time,event_name,user_id,source,event_time_rounded,time_diff,time_diff_minutes,session_id
805,2019-10-07 13:39:45.989359,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:39:46,NaT,NaN,1
806,2019-10-07 13:40:31.052909,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:40:31,0 days 00:00:45.063550,0.8,1
809,2019-10-07 13:41:05.722489,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:41:06,0 days 00:00:34.669580,0.6,1
820,2019-10-07 13:43:20.735461,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:43:21,0 days 00:02:15.012972,2.3,1
830,2019-10-07 13:45:30.917502,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:45:31,0 days 00:02:10.182041,2.2,1
831,2019-10-07 13:45:43.212340,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:45:43,0 days 00:00:12.294838,0.2,1
832,2019-10-07 13:46:31.033718,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:46:31,0 days 00:00:47.821378,0.8,1
836,2019-10-07 13:47:32.860234,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:47:33,0 days 00:01:01.826516,1.0,1
839,2019-10-07 13:49:41.716617,tips_show,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-07 13:49:42,0 days 00:02:08.856383,2.1,1
6541,2019-10-09 18:33:55.577963,map,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,other,2019-10-09 18:33:56,2 days 04:44:13.861346,"3,164.2",2


Когда значение интервала бездействия > выбранного нами, счётчик сессий увеличивается

<a id="path"></a>
## Поиск путей (сценариев), приводящих к целевому действию

Избавимся от дубликатов последовательных событий в сессиях

In [20]:
sorted_data.shape

(73079, 8)

In [21]:
# убираем повторяющиеся события внутри сессий пользователя
deduplicated = sorted_data.drop_duplicates(subset=['session_id', 'event_name', 'user_id'], keep='first')

In [22]:
deduplicated.shape

(23519, 8)

Датафрейм уменьшился до трети значений

Найдём популярные пути пользователей, в которых присутствует `contacts_show`

In [23]:
contacts_show_sessions = deduplicated[deduplicated['event_name'] == 'contacts_show']['session_id'].unique()

# группируем по сессии и склеиваем названия событий
df_grouped = deduplicated[deduplicated['session_id'].isin(contacts_show_sessions)].groupby('session_id')['event_name'].apply(lambda x: ' -> '.join(x)).reset_index()
grouped = df_grouped.copy()

# выведем топ 20
grouped.groupby('event_name').agg(count=('session_id','count')).sort_values(by='count', ascending=False).head(20)

,count
event_name,
tips_show -> contacts_show,482
contacts_show,389
contacts_show -> contacts_call,160
contacts_show -> tips_show,134
photos_show -> contacts_show,98
map -> tips_show -> contacts_show,81
search_1 -> contacts_show -> contacts_call,55
search_1 -> contacts_show,47
photos_show -> contacts_show -> contacts_call,40


Наиболее популярные пути:

- tips_show -> contacts_show
- photos_show -> contacts_show	
- map -> tips_show -> contacts_show
- map -> contacts_show
- search_1 -> contacts_show	
- search_1 -> photos_show -> contacts_show
- advert_open -> contacts_show
- advert_open -> tips_show -> contacts_show

Построим диаграмму санкей

In [24]:
def add_features(df):
    
    """Функция генерации новых столбцов для исходной таблицы

    Args:
        df (pd.DataFrame): исходная таблица.
    Returns:
        pd.DataFrame: таблица с новыми признаками.
    """
    
    # сортируем по id и времени
    sorted_df = df.sort_values(by=['session_id', 'event_time']).copy()
    # добавляем шаги событий
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    
    # добавляем узлы-источники и целевые узлы
    # узлы-источники - это сами события
    sorted_df['source_event'] = sorted_df['event_name']
    # добавляем целевые узлы
    sorted_df['target_event'] = sorted_df.groupby('session_id')['source_event'].shift(-1)
    
    # возврат таблицы без имени событий
    return sorted_df.drop(['event_name'], axis=1)
  
# преобразуем таблицу
table = add_features(deduplicated[['user_id', 'event_time', 'event_name', 'session_id', 'source']])
table.head()

,user_id,event_time,session_id,source,step,source_event,target_event
805,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-07 13:39:45.989359,1,other,1,tips_show,NaN
6541,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-09 18:33:55.577963,2,other,1,map,tips_show
6565,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-09 18:40:28.738785,2,other,2,tips_show,NaN
36412,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-21 19:52:30.778932,3,other,1,tips_show,map
36419,0001b1d5-b74a-4cbf-aeb0-7df5947bf349,2019-10-21 19:53:38.767230,3,other,2,map,NaN


In [25]:
# удалим все пары source-target, шаг которых превышает 7
# и сохраним полученную таблицу в отдельную переменную
df_comp = table[table['step'] <= 7].copy().reset_index(drop=True)
df_comp.shape

(23514, 7)

In [26]:
def get_source_index(df):
    
    """Функция генерации индексов source_event

    Args:
        df (pd.DataFrame): исходная таблица с признаками step, source_event, target.
    Returns:
        dict: словарь с индексами, именами и соответсвиями индексов именам source_event.
    """
    
    res_dict = {}
    
    count = 0
    # получаем индексы источников
    for no, step in enumerate(df['step'].unique().tolist()):
        # получаем уникальные наименования для шага
        res_dict[no+1] = {}
        res_dict[no+1]['sources'] = df[df['step'] == step]['source_event'].unique().tolist()
        res_dict[no+1]['sources_index'] = []
        for i in range(len(res_dict[no+1]['sources'])):
            res_dict[no+1]['sources_index'].append(count)
            count += 1
            
    # соединим списки
    for key in res_dict:
        res_dict[key]['sources_dict'] = {}
        for name, no in zip(res_dict[key]['sources'], res_dict[key]['sources_index']):
            res_dict[key]['sources_dict'][name] = no
    return res_dict
  

# создаем словарь
source_indexes = get_source_index(df_comp)

In [27]:
def generate_random_color():
    
    """Случайная генерация цветов rgba

    Args:
        
    Returns:
        str: Строка со сгенерированными параметрами цвета
    """
    
    # сгенерим значение для каждого канала
    r, g, b = np.random.randint(255, size=3)
    return f'rgba({r}, {g}, {b}, 1)'

In [28]:
def colors_for_sources(mode):
    
    """Генерация цветов rgba

    Args:
        mode (str): сгенерировать случайные цвета, если 'random', а если 'custom' - 
                    использовать заранее подготовленные
    Returns:
        dict: словарь с цветами, соответствующими каждому индексу
    """
    # словарь, в который сложим цвета в соответствии с индексом
    colors_dict = {}
    
    if mode == 'random':
        # генерим случайные цвета
        for label in df_comp['source_event'].unique():
            r, g, b = np.random.randint(255, size=3)            
            colors_dict[label] = f'rgba({r}, {g}, {b}, 1)'
            
            
    return colors_dict
  
  
# генерю цвета из своего списка
colors_dict = colors_for_sources(mode='random')

In [29]:
def percent_users(sources, targets, values):
    
    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)
    
    Args:
        sources (list): список с индексами source_event.
        targets (list): список с индексами target_event.
        values (list): список с "объемами" потоков.
        
    Returns:
        list: список с "объемами" потоков в процентах
    """
    
    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))
    
    new_list = []
    
    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}
    
    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl
                    
    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))
    
    return new_list

In [30]:
def lists_for_plot(source_indexes=source_indexes, colors=colors_dict, frac=10):
    
    """
    Создаем необходимые для отрисовки диаграммы переменные списков и возвращаем
    их в виде словаря
    
    Args:
        source_indexes (dict): словарь с именами и индексами source.
        colors (dict): словарь с цветами source.
        frac (int): ограничение на минимальный "объем" между узлами.
        
    Returns:
        dict: словарь со списками, необходимыми для диаграммы.
    """
    
    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []

    # проходим по каждому шагу
    for step in df_comp['step'].unique():
        if step + 1 not in source_indexes:
            continue

        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']

        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in temp_dict_source.items():
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id            
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source_event'] == source)&(df_comp['target_event'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))
                    
    labels = []
    colors_labels = []
    for key in source_indexes:
        for name in source_indexes[key]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])
            
    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)
    
    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")
    
    # возвратим словарь с вложенными списками
    return {'sources': sources, 
            'targets': targets, 
            'values': values, 
            'labels': labels, 
            'colors_labels': colors_labels, 
            'link_color': link_color, 
            'link_text': link_text}
  

# создаем словарь
data_for_plot = lists_for_plot()

In [31]:
def plot_senkey_diagram(data_dict=data_for_plot):    
    
    """
    Функция для генерации объекта диаграммы Сенкей 
    
    Args:
        data_dict (dict): словарь со списками данных для построения.
        
    Returns:
        plotly.graph_objs._figure.Figure: объект изображения.
    """
    
    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'],
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Sankey Diagram", font_size=10)
    
    # возвращаем объект диаграммы
    return fig
  

# сохраняем диаграмму в переменную
senkey_diagram = plot_senkey_diagram()

In [32]:
senkey_diagram.show()

Диаграмма показывает те же основные пути пользователей:
- `coontacts_show` на втором уровне:
  - tips_show -> contacts_show
  - photos_show -> contacts_show	
  - map -> contacts_show
  - search_1 -> contacts_show	
  - advert_open -> contacts_show
  - favourites_add -> contacts_show
- `coontacts_show` на третьем уровне:
  - map -> tips_show -> contacts_show
  - search_1 -> photos_show -> contacts_show
  - advert_open -> tips_show -> contacts_show

<a id="conversion"></a>
## Рассчёт воронок и конверсии

Выберем основные воронки:
- tips_show -> contacts_show
- photos_show -> contacts_show	
- map -> tips_show -> contacts_show
- map -> contacts_show
- search_1 -> contacts_show	
- search_1 -> photos_show -> contacts_show
- advert_open -> contacts_show
- advert_open -> tips_show -> contacts_show

In [33]:
def show_funnel(stages, df):
    # Define your funnel stages
    stages = stages

    # Count unique users at each stage
    funnel_data = []
    stage_data = df
    for stage in stages:
        # Уникальные пользователи на этом шаге
        stage_users = stage_data[stage_data['event_name'] == stage]['user_id'].unique()
        # количество уникальных пользователей на шаге
        count = stage_data[stage_data['event_name'] == stage]['user_id'].nunique()
        # фильтруем данные для следующего шага
        stage_data = stage_data[stage_data['user_id'].isin(stage_users)]
        # добавляем шаг воронки
        funnel_data.append({'Stage': stage, 'Number': count})

    # Создаём датафрейм для воронки
    funnel_df = pd.DataFrame(funnel_data)
    
    fig = go.Figure(go.Funnel(
        x=funnel_df['Number'], y=funnel_df['Stage'],
        textposition = "inside",
        textinfo = "value+percent initial")
    )
    
    return fig

Выведем воронки по интересующим нас путям

In [34]:
# создадим словарь воронок
funnel_dict = [
    ['tips_show', 'contacts_show'],
    ['photos_show', 'contacts_show'],
    ['map', 'tips_show', 'contacts_show'],
    ['map', 'contacts_show'],
    ['search_1', 'contacts_show'],
    ['search_1', 'photos_show', 'contacts_show'],
    ['advert_open', 'contacts_show'],
    ['advert_open', 'tips_show', 'contacts_show']
]

In [35]:
for elem in funnel_dict:
    fig = show_funnel(elem, deduplicated)

    fig.update_layout(
        title=f'Пользовательская воронка {elem}',
        yaxis_title='Название события'
    )

    # Show the plot
    fig.show()

**Промежуточные итоги:**

- `tips_show` -> `contacts_show` - конверсия 18%, 516 пользователей
  - средняя конверсия, эта воронка может являться частью других путей, если у нас недостаточно данных
- `photos_show` -> `contacts_show` - конверсия 31%, 339 пользователей
  - высокая конверсия
- `map` -> `tips_show` -> `contacts_show` - конверсия 19%, 275 пользователей
  - средняя конверсия
- `map` -> `contacts_show` - конверсия 20%, 289 пользователей
  - конверсия средняя, чуть выше чем в таком же пути с `tips_show`
- `search_1` -> `contacts_show` - конверсия 30%, 237 пользователей
  - высокая конверсия
- `search_1` -> `photos_show` -> `contacts_show` - конверсия 24%, 191 пользователей
  - конверсия выше среднего, но ниже, чем в таком же пути без `photos_show`
- `advert_open` -> `contacts_show` - конверсия 18%, 138 пользователей
  - средняя конверсия
- `advert_open` -> `tips_show` -> `contacts_show` - конверсия 12%, 88 пользователей
  - конверсия ниже, чем в таком же пути без `tips_show`

Гипотезы:
- пользователи чаще смотрят контакты в объявлениях с фотографиями
  - однако если пользователи пришли из поиска, в объявлениях с фотографиями реже просматривают контакты, чем в объявлениях без фотографий
- `tips_show` (рекомендации) в некоторых путях портят конверсию
- пользователи, пришедшие из поиска, часто просматривают контакты

<a id="frequency"></a>
## Проверка относительной частоты событий в разрезе групп пользователей

Разделим пользователей на группы (добавим столбец с признаком)

Пользаватели, которые видели контакты `contacts_show`

In [36]:
contacts_show_users = data[data['event_name'] == 'contacts_show']['user_id'].unique()

data['shown'] = data['user_id'].isin(contacts_show_users)
data[data['shown']==True]['event_name'].value_counts(normalize=True)*100

event_name
tips_show       47.7
contacts_show   16.5
photos_show     13.1
advert_open      6.0
search_1         5.0
map              4.0
contacts_call    2.0
favorites_add    1.6
tips_click       1.2
search_5         0.9
search_4         0.6
search_3         0.5
search_2         0.4
search_6         0.3
search_7         0.1
Name: proportion, dtype: float64

Пользаватели, которые НЕ видели контакты `contacts_show`

In [37]:
data[data['shown']==False]['event_name'].value_counts(normalize=True)*100

event_name
tips_show       58.5
photos_show     12.6
advert_open      9.8
map              5.8
search_1         4.6
favorites_add    2.1
search_5         1.7
search_4         1.2
tips_click       1.0
search_6         0.8
search_3         0.8
search_2         0.5
search_7         0.4
Name: proportion, dtype: float64

Сгруппируем по признаку видели / не видели для визуализации

In [38]:
grouped_data = data.groupby(['shown'])['event_name'].value_counts(normalize=True)*100
grouped_data = grouped_data.reset_index(name='Percentage')

In [39]:
# Фильтруем данные по признаку видели / не видели
true_data = grouped_data[grouped_data['shown'] == True]
false_data = grouped_data[grouped_data['shown'] == False]

trace_true = go.Bar(x=true_data['event_name'], y=true_data['Percentage'], name='Видели контакты')
trace_false = go.Bar(x=false_data['event_name'], y=false_data['Percentage'], name='Не видели контакты')

# Создаём графики
fig = go.Figure(data=[trace_true, trace_false])

# Задаём подписи
fig.update_layout(
    title='Относительная частота событий',
    xaxis_title='Название события',
    yaxis_title='Частота (%)',
    barmode='group'  
)

# Отображаем
fig.show()

Наиболее частые события в обеих группах - `tips_show`. Чаще оно встречается в группе, которые контакты не видели (подтверждает гипотезу отрицательного влияния).
Также у пользователей, которые не видели контакты, относительно чаще `advert_open`, `map`, `favorites_add`. На `map` и `advert_open` судя по воронкам тоже негативно влияет `tips_show`.

<a id="hypothesys"></a>
# Проверка гипотез

<a id="h1"></a>
## Гипотеза 1

**Исследуемая Гипотеза:**
"Конверсия в просмотры контактов различается между пользователями, которые совершают действия `tips_show` и `tips_click`, и теми, кто совершает только `tips_show`".

- **Нулевая Гипотеза (H0)**: Нет статистически значимой разницы в конверсии в просмотры контактов между группами пользователей, которые совершают `tips_show` и `tips_click`, и теми, кто совершает только `tips_show`.
- **Альтернативная Гипотеза (H1)**: Существует статистически значимая разница в конверсии в просмотры контактов между этими двумя группами пользователей.

In [40]:
# Фильтруем интересующие нас действия
actions_of_interest = ['tips_show', 'tips_click', 'contacts_show']
filtered_data = data[data['event_name'].isin(actions_of_interest)]

# те, кто совершал 'tips_click'
users_clicked_tips = filtered_data[filtered_data['event_name'] == 'tips_click']['user_id'].unique()

# делим данные на группы
group_a = filtered_data[filtered_data['user_id'].isin(users_clicked_tips)]
group_b = filtered_data[~filtered_data['user_id'].isin(users_clicked_tips)]

# Рассчёт конверсии
def calculate_conversion_rate(group):
    total_users = group['user_id'].nunique()
    users_viewed_contacts = group[group['event_name'] == 'contacts_show']['user_id'].nunique()
    return users_viewed_contacts / total_users

# Рассчёт конверсии для каждой группы
conversion_rate_a = calculate_conversion_rate(group_a)
conversion_rate_b = calculate_conversion_rate(group_b)

# Данные для T-теста
group_a_conversions = group_a.groupby('user_id')['event_name'].apply(lambda x: 1 if 'contacts_show' in x.values else 0)
group_b_conversions = group_b.groupby('user_id')['event_name'].apply(lambda x: 1 if 'contacts_show' in x.values else 0)

# Т-тест
t_stat, p_value = stats.ttest_ind(group_a_conversions, group_b_conversions, equal_var=False)

print(f'P-value: {p_value}')

# Интерпретация p-value
alpha = 0.05
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: существует значительная разница в коэффициентах конверсии.")
else:
    print("Невозможно отвергнуть нулевую гипотезу: существенной разницы в коэффициентах конверсии нет.")




P-value: 0.6344515656214196
Невозможно отвергнуть нулевую гипотезу: существенной разницы в коэффициентах конверсии нет.


Не смогли отвергнуть нулевую гипотезу. Это означает, что на основании проведённого анализа данных нет достаточных статистических доказательств для утверждения о том, что конверсия в просмотры контактов различается между пользователями, совершающими действия `tips_show` и `tips_click`, и теми, кто совершает только `tips_show`.

Это не обязательно означает, что `tips_show` и `tips_click` не влияют на просмотр контактов.
- Возможно, у нас недостаточно данных для обнаружения разницы, если она существует.
- Нет значимого влияния: `tips_show` и `tips_click` действительно могут не оказывать значительного влияния на просмотр контактов. То есть пользователи, которые совершают только `tips_show`, могут быть столь же склонны к просмотру контактов, как и те, кто также совершает `tips_click`. 
- `tips_click` не добавляет дополнительной ценности: Возможно, `tips_click` не добавляет значимой ценности к взаимодействию пользователя с `tips_show` в контексте просмотра контактов. Это может означать, что простой просмотр рекомендаций (`tips_show`) уже достаточен для пользователя, чтобы решить, интересен ли ему контакт, и дополнительное действие (`tips_click`) не требуется. Возможно, для просмотра контактов не требуется дополнительных действий.

<a id="h2"></a>
## Гипотеза 2

**Исследуемая Гипотеза:**
"Пользователи, пришедшие из источника Google, демонстрируют более высокую конверсию в `contacts_show` по сравнению с пользователями из других источников".

- **Нулевая Гипотеза (H0)**: Нет статистически значимой разницы в конверсии в `contacts_show` между пользователями из источника Google и пользователями из других источников.
- **Альтернативная Гипотеза (H1)**: Пользователи из источника Google имеют статистически значимо более высокую конверсию в `contacts_show` по сравнению с пользователями из других источников.

Проведём Z-тест

In [41]:
# Выбираем только интересующие нас события
data_of_interest = data[data['event_name'] == 'contacts_show']

# Делим пользователей на группы по источнику
group_google = data_of_interest[data_of_interest['source'] == 'google']
group_others = data_of_interest[data_of_interest['source'] != 'google']

# Расчет доли пользователей, которые посмотрели контакты
conversion_google = group_google['user_id'].nunique() / data[data['source'] == 'google']['user_id'].nunique()
conversion_others = group_others['user_id'].nunique() / data[data['source'] != 'google']['user_id'].nunique()

# Расчет успехов и общего количества наблюдений
successes_google = group_google['user_id'].nunique()
successes_others = group_others['user_id'].nunique()

n_google = data[data['source'] == 'google']['user_id'].nunique()
n_others = data[data['source'] != 'google']['user_id'].nunique()

# Выполнение Z-теста
count = np.array([successes_google, successes_others])
nobs = np.array([n_google, n_others])
stat, pval = proportions_ztest(count, nobs)

print(f'P-value: {pval}')

# Интерпретация p-значения
alpha = 0.05
if pval < alpha:
    print("Отвергаем нулевую гипотезу: существует значимая разница в конверсии.")
else:
    print("Не отвергаем нулевую гипотезу: нет значимых различий в конверсии.")


P-value: 0.16017834813412768
Не отвергаем нулевую гипотезу: нет значимых различий в конверсии.


<a id="conclusion"></a>
# Выводы и рекомендации

Напомним цель: *Получить на основе поведения пользователей гипотезы о том как можно было бы улучшить приложение с точки зрения пользовательского опыта.*

1. Мы проанализировали связь целевого события — просмотра контактов — и других действий пользователей.
- В разрезе сессий отобрали сценарии\паттерны, которые приводят к просмотру контактов.
- Построили воронки по основным сценариям в разрезе уникальных пользователей:

- `tips_show` -> `contacts_show` - конверсия 18%, 516 пользователей
  - средняя конверсия, эта воронка может являться частью других путей, если у нас недостаточно данных
- `photos_show` -> `contacts_show` - конверсия 31%, 339 пользователей
  - высокая конверсия
- `map` -> `tips_show` -> `contacts_show` - конверсия 19%, 275 пользователей
  - средняя конверсия
- `map` -> `contacts_show` - конверсия 20%, 289 пользователей
  - конверсия средняя, чуть выше чем в таком же пути с `tips_show`
- `search_1` -> `contacts_show` - конверсия 30%, 237 пользователей
  - высокая конверсия
- `search_1` -> `photos_show` -> `contacts_show` - конверсия 24%, 191 пользователей
  - конверсия выше среднего, но ниже, чем в таком же пути без `photos_show`
- `advert_open` -> `contacts_show` - конверсия 18%, 138 пользователей
  - средняя конверсия
- `advert_open` -> `tips_show` -> `contacts_show` - конверсия 12%, 88 пользователей
  - конверсия ниже, чем в таком же пути без `tips_show`


2. Оценили, какие действия чаще совершают те пользователи, которые просматривают контакты.

Наиболее частые события в обеих группах - `tips_show`. Чаще оно встречается в группе, которые контакты не видели (подтверждает гипотезу отрицательного влияния).
Также у пользователей, которые не видели контакты, относительно чаще `advert_open`, `map`, `favorites_add`. На `map` и `advert_open` судя по воронкам тоже негативно влияет `tips_show`.

3. На основании проведённого анализа данных нет достаточных статистических доказательств для утверждения о том, что конверсия в просмотры контактов различается между пользователями, совершающими действия `tips_show` и `tips_click`, и теми, кто совершает только `tips_show`.

- Возможно, у нас недостаточно данных для обнаружения разницы, если она существует.
- Нет значимого влияния: `tips_show` и `tips_click` действительно могут не оказывать значительного влияния на просмотр контактов. То есть пользователи, которые совершают только `tips_show`, могут быть столь же склонны к просмотру контактов, как и те, кто также совершает `tips_click`. 
- `tips_click` не добавляет дополнительной ценности: Возможно, `tips_click` не добавляет значимой ценности к взаимодействию пользователя с `tips_show` в контексте просмотра контактов. Это может означать, что простой просмотр рекомендаций (`tips_show`) уже достаточен для пользователя, чтобы решить, интересен ли ему контакт, и дополнительное действие (`tips_click`) не требуется. Возможно, для просмотра контактов не требуется дополнительных действий.

1. **Ключевые Пути Просмотра Контактов:** Несколько путей пользователя ведут к просмотру контактов, при этом коэффициенты конверсии различаются. Пути, включающие `photos_show` и `search_1`, непосредственно ведущие к `contacts_show`, имеют высокие коэффициенты конверсии, что указывает на особую эффективность этих функций в привлечении внимания пользователей к просмотру контактов.
2. **Влияние `tips_show`:** Присутствие `tips_show` в пути пользователя имеет смешанное влияние на коэффициенты конверсии. В некоторых случаях это улучшает путь к просмотру контактов, а в других, кажется, снижает коэффициент конверсии.
3. **Различия в Поведении Пользователей:** Пользователи, просматривающие контакты, ведут себя по-другому по сравнению с теми, кто их не просматривает. Например, событие `tips_show` чаще встречается среди пользователей, которые не просматривают контакты, что может указывать на его менее эффективное использование или даже отвлекающий характер в некоторых контекстах.

### Рекомендации:
1. **Улучшение Функциональности Фотографий:** Учитывая высокий коэффициент конверсии путей, включающих `photos_show`, стоит уделить внимание улучшению этой функции. Это может включать повышение качества фотографий, упрощение навигации по фотографиям или более заметное отображение фотографий в объявлениях.
2. **Оптимизация Функциональности Поиска:** Учитывая высокую конверсию из пути `search_1`, следует сосредоточиться на оптимизации функции поиска. Это может включать улучшение алгоритмов поиска, пользовательского интерфейса или релевантности результатов поиска.
3. **Переоценка Использования `tips_show`:** Поскольку `tips_show` оказывает смешанное влияние, стоит исследовать, как оно представлено и когда запускается. A/B-тестирование может быть использовано для нахождения наиболее эффективного способа интеграции этой функции без негативного влияния на пользовательский опыт.
4. **Целевые Улучшения Пользовательского Опыта:** Адаптируйте приложение с учетом поведения пользователей. Для пользователей, которые часто используют карту или рекламные объявления, упростите эти функции для более эффективного привлечения к просмотру контактов.
5. **Анализ Пути Пользователя:** Проведите дополнительный анализ причин, по которым некоторые пути имеют более низкий коэффициент конверсии. Например, почему добавление `photos_show` в путь поиска снижает просмотр контактов?


## Презентация